In [1]:
import os

In [2]:
os.chdir('../../')

In [3]:
%pwd

'/Users/bhikipallai/Desktop/Projects/95Mobiles'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class mobile_datasplitconfig:
    root_dir: Path
    X_train: Path
    X_test: Path
    y_train:Path
    y_test: Path
    final_data: Path
    test_size: float
    random_state: int

In [5]:
from src.mobiles.constants import *
from src.mobiles.utils.common import read_yaml,create_directory

[2024-10-06 18:06:17,346: INFO]: Logging file start


In [12]:
class ConfigurationManger:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH) :
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.data_root])

    def get_data_split(self) -> mobile_datasplitconfig:
        config = self.config.data_split
        params = self.params.split

        create_directory([config.root_dir])

        model_datasplit = mobile_datasplitconfig(
            root_dir=config.root_dir,
            X_train=config.X_train,
            X_test=config.X_test,
            y_train=config.y_train,
            y_test=config.y_test,
            test_size=params.test_size,
            random_state=params.random_state,
            final_data=config.final_data
        )
        return model_datasplit

In [13]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [16]:
class mobile_datasplit:
    def __init__(self,config:mobile_datasplitconfig):
        self.config = config
    
    def load_data(self):
        self.df = pd.read_csv(self.config.final_data)
        return self.df
    
    def split_data(self):
        df = self.df
        X =df.drop('price',axis=1)
        y=df['price']
        self.X_train,self.X_test,self.y_train,self.y_test = train_test_split(X,y,test_size=self.config.test_size,random_state=self.config.random_state)
        return self.X_train,self.X_test,self.y_train,self.y_test
    
    def save_data(self):
        X_train = self.X_train
        X_test = self.X_test
        y_train = self.y_train
        y_test = self.y_test

        X_train.to_csv(self.config.X_train,index=False)
        X_test.to_csv(self.config.X_test,index = False)
        y_train.to_csv(self.config.y_train,index=False)
        y_test.to_csv(self.config.y_test,index=False)
        print(X_train.head(2))
        print(X_test.head(2))


In [17]:
try:
    config = ConfigurationManger()
    split = config.get_data_split()
    ds = mobile_datasplit(config=split)
    ds.load_data()
    ds.split_data()
    ds.save_data()
except Exception as e:
    raise e

[2024-10-06 18:14:33,493: INFO]: yaml: config/mobiles_config.yaml loaded successfully
[2024-10-06 18:14:33,495: INFO]: yaml: params/mobiles_params.yaml loaded successfully
[2024-10-06 18:14:33,496: INFO]: created directory at: data
[2024-10-06 18:14:33,498: INFO]: created directory at: data/mobile/data_split
     Brand  Model   Storage       RAM  screen_size  camera   battery
198      7    139 -0.910044 -0.756626           20      43  1.662271
243     14    215  0.076342  0.066761           27      20  0.406327
     Brand  Model   Storage       RAM  screen_size  camera   battery
192     12     68 -0.910044 -0.756626           20      67  0.406327
319     15    169  0.076342 -0.756626           20      71  0.406327
